# CYGNSS Monthly Average Processing

Notebook này xử lý dữ liệu CYGNSS hàng ngày để tạo ra các file trung bình hàng tháng.

## Mô tả dữ liệu
- **Nguồn**: CYGNSS L3 Grid Soil Moisture 36km
- **Độ phân giải**: 36km
- **Thời gian**: 2022-2024 (dữ liệu hàng ngày)
- **Format file**: NetCDF (.nc)

## Quy trình
1. Đọc và phân tích cấu trúc dữ liệu
2. Nhóm các file theo tháng/năm
3. Tính trung bình hàng tháng
4. Lưu kết quả vào file NetCDF mới

---

In [15]:
# Import các thư viện cần thiết
import os
import glob
import numpy as np
import pandas as pd
import xarray as xr
from datetime import datetime, timedelta
import re
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("Đã import các thư viện thành công!")

Đã import các thư viện thành công!


In [16]:
# Thiết lập đường dẫn
data_folder = "36km"  # Thư mục chứa dữ liệu CYGNSS (cùng cấp với notebook này)
output_folder = "monthly_averages"  # Thư mục xuất file trung bình hàng tháng

# Tạo thư mục output nếu chưa có
os.makedirs(output_folder, exist_ok=True)

# Kiểm tra đường dẫn dữ liệu
if os.path.exists(data_folder):
    print(f"✓ Thư mục dữ liệu tồn tại: {data_folder}")
    
    # Đếm số file .nc
    nc_files = glob.glob(os.path.join(data_folder, "*.nc"))
    print(f"✓ Tìm thấy {len(nc_files)} file NetCDF")
    
    if len(nc_files) > 0:
        print(f"✓ File đầu tiên: {os.path.basename(nc_files[0])}")
        print(f"✓ File cuối cùng: {os.path.basename(nc_files[-1])}")
else:
    print(f"✗ Không tìm thấy thư mục: {data_folder}")

print(f"✓ Thư mục output: {output_folder}")

✓ Thư mục dữ liệu tồn tại: 36km
✓ Tìm thấy 1096 file NetCDF
✓ File đầu tiên: cyg.ddmi.s20220101-030000-e20220101-210000.l3.grid-soil-moisture-36km.a32.d33.nc
✓ File cuối cùng: cyg.ddmi.s20241231-030000-e20241231-210000.l3.grid-soil-moisture-36km.a32.d33.nc
✓ Thư mục output: monthly_averages


In [17]:
def extract_date_from_filename(filename):
    """
    Trích xuất thông tin ngày từ tên file CYGNSS
    
    Input: cyg.ddmi.s20220101-030000-e20220101-210000.l3.grid-soil-moisture-36km.a32.d33.nc
    Output: datetime object
    """
    # Sử dụng regex để tìm ngày trong format s20220101
    pattern = r's(\d{8})-'
    match = re.search(pattern, filename)
    
    if match:
        date_str = match.group(1)  # 20220101
        try:
            date_obj = datetime.strptime(date_str, '%Y%m%d')
            return date_obj
        except ValueError:
            print(f"Lỗi chuyển đổi ngày cho file: {filename}")
            return None
    else:
        print(f"Không tìm thấy pattern ngày trong file: {filename}")
        return None

def get_month_year_key(date_obj):
    """
    Tạo key cho tháng-năm (format: YYYY-MM)
    """
    if date_obj:
        return date_obj.strftime('%Y-%m')
    return None

# Test function với file mẫu
test_filename = "cyg.ddmi.s20220101-030000-e20220101-210000.l3.grid-soil-moisture-36km.a32.d33.nc"
test_date = extract_date_from_filename(test_filename)
print(f"Test filename: {test_filename}")
print(f"Extracted date: {test_date}")
print(f"Month-Year key: {get_month_year_key(test_date)}")

Test filename: cyg.ddmi.s20220101-030000-e20220101-210000.l3.grid-soil-moisture-36km.a32.d33.nc
Extracted date: 2022-01-01 00:00:00
Month-Year key: 2022-01


In [18]:
# Phân tích và nhóm files theo tháng
def organize_files_by_month(data_folder):
    """
    Nhóm các file theo tháng-năm
    """
    nc_files = glob.glob(os.path.join(data_folder, "*.nc"))
    monthly_files = {}
    
    print(f"Đang phân tích {len(nc_files)} files...")
    
    for file_path in nc_files:
        filename = os.path.basename(file_path)
        date_obj = extract_date_from_filename(filename)
        
        if date_obj:
            month_key = get_month_year_key(date_obj)
            if month_key not in monthly_files:
                monthly_files[month_key] = []
            monthly_files[month_key].append(file_path)
    
    # Sắp xếp theo thứ tự thời gian
    sorted_months = sorted(monthly_files.keys())
    
    print(f"\n📊 Tóm tắt dữ liệu:")
    print(f"Tổng số tháng: {len(sorted_months)}")
    
    if len(sorted_months) > 0:
        print(f"Từ: {sorted_months[0]} đến: {sorted_months[-1]}")
        
        # Hiển thị thống kê chi tiết theo năm
        print(f"\n📈 Số file theo từng năm:")
        
        # Nhóm theo năm
        years = {}
        for month in sorted_months:
            year = month.split('-')[0]
            if year not in years:
                years[year] = []
            years[year].append(month)
        
        for year in sorted(years.keys()):
            months_in_year = years[year]
            total_files = sum(len(monthly_files[month]) for month in months_in_year)
            print(f"  {year}: {len(months_in_year)} tháng, {total_files} files")
            
            # Hiển thị chi tiết các tháng của năm này
            for month in months_in_year:
                count = len(monthly_files[month])
                print(f"    {month}: {count} files")
    else:
        print("❌ Không tìm thấy file nào hoặc không thể trích xuất thông tin thời gian!")
        print("🔍 Kiểm tra:")
        print("- Đường dẫn thư mục dữ liệu có đúng không?")
        print("- Có file .nc nào trong thư mục không?")
        print("- Tên file có đúng định dạng CYGNSS không?")
    
    return monthly_files, sorted_months

# Thực hiện phân tích
monthly_files_dict, sorted_months = organize_files_by_month(data_folder)

Đang phân tích 1096 files...

📊 Tóm tắt dữ liệu:
Tổng số tháng: 36
Từ: 2022-01 đến: 2024-12

📈 Số file theo từng năm:
  2022: 12 tháng, 365 files
    2022-01: 31 files
    2022-02: 28 files
    2022-03: 31 files
    2022-04: 30 files
    2022-05: 31 files
    2022-06: 30 files
    2022-07: 31 files
    2022-08: 31 files
    2022-09: 30 files
    2022-10: 31 files
    2022-11: 30 files
    2022-12: 31 files
  2023: 12 tháng, 365 files
    2023-01: 31 files
    2023-02: 28 files
    2023-03: 31 files
    2023-04: 30 files
    2023-05: 31 files
    2023-06: 30 files
    2023-07: 31 files
    2023-08: 31 files
    2023-09: 30 files
    2023-10: 31 files
    2023-11: 30 files
    2023-12: 31 files
  2024: 12 tháng, 366 files
    2024-01: 31 files
    2024-02: 29 files
    2024-03: 31 files
    2024-04: 30 files
    2024-05: 31 files
    2024-06: 30 files
    2024-07: 31 files
    2024-08: 31 files
    2024-09: 30 files
    2024-10: 31 files
    2024-11: 30 files
    2024-12: 31 files


In [19]:
# Kiểm tra cấu trúc dữ liệu trong file NetCDF
def examine_netcdf_structure(file_path):
    """
    Kiểm tra cấu trúc dữ liệu NetCDF
    """
    print(f"📁 Kiểm tra file: {os.path.basename(file_path)}")
    
    try:
        with xr.open_dataset(file_path) as ds:
            print(f"\n📏 Kích thước dataset:")
            for dim, size in ds.dims.items():
                print(f"  {dim}: {size}")
            
            print(f"\n📊 Các biến dữ liệu:")
            for var_name, var in ds.data_vars.items():
                print(f"  {var_name}: {var.dims} - {var.dtype}")
                if hasattr(var, 'long_name'):
                    print(f"    Mô tả: {var.long_name}")
                if hasattr(var, 'units'):
                    print(f"    Đơn vị: {var.units}")
            
            print(f"\n📍 Tọa độ:")
            for coord_name, coord in ds.coords.items():
                print(f"  {coord_name}: {coord.dims} - {coord.dtype}")
                if coord_name in ['lat', 'latitude', 'lon', 'longitude']:
                    print(f"    Range: {float(coord.min()):.3f} to {float(coord.max()):.3f}")
            
            return ds
            
    except Exception as e:
        print(f"❌ Lỗi đọc file: {e}")
        return None

# Kiểm tra file đầu tiên
if sorted_months and monthly_files_dict:
    first_month = sorted_months[0]
    first_file = monthly_files_dict[first_month][0]
    sample_ds = examine_netcdf_structure(first_file)

📁 Kiểm tra file: cyg.ddmi.s20220101-030000-e20220101-210000.l3.grid-soil-moisture-36km.a32.d33.nc

📏 Kích thước dataset:
  time: 1
  lat: 252
  lon: 802
  timeslices: 4
  startstop: 2

📊 Các biến dữ liệu:
  SM_daily: ('time', 'lat', 'lon') - float32
    Mô tả: Daily Soil Moisture
    Đơn vị: cm3 cm-3
  SM_subdaily: ('timeslices', 'lat', 'lon') - float32
    Mô tả: Sub-daily Soil Moisture
    Đơn vị: cm3 cm-3
  SIGMA_daily: ('time', 'lat', 'lon') - float32
    Mô tả: Daily Soil Moisture Sigma
    Đơn vị: cm3 cm-3
  SIGMA_subdaily: ('timeslices', 'lat', 'lon') - float32
    Mô tả: Sub-daily Soil Moisture Sigma
    Đơn vị: cm3 cm-3
  timeintervals: ('timeslices', 'startstop') - datetime64[ns]
    Mô tả: start and stop time for the sub-daily time periods

📍 Tọa độ:
  time: ('time',) - datetime64[ns]
  latitude: ('lat', 'lon') - float32

📏 Kích thước dataset:
  time: 1
  lat: 252
  lon: 802
  timeslices: 4
  startstop: 2

📊 Các biến dữ liệu:
  SM_daily: ('time', 'lat', 'lon') - float32
    

In [20]:
def calculate_monthly_average(file_list, month_key, output_folder):
    """
    Tính trung bình hàng tháng từ danh sách files
    
    Parameters:
    - file_list: Danh sách đường dẫn file NetCDF
    - month_key: Key tháng (YYYY-MM)
    - output_folder: Thư mục lưu kết quả
    """
    print(f"\n🔄 Đang xử lý tháng {month_key} ({len(file_list)} files)...")
    
    try:
        # Đọc tất cả files cùng lúc
        datasets = []
        valid_files = []
        
        for file_path in file_list:
            try:
                ds = xr.open_dataset(file_path)
                datasets.append(ds)
                valid_files.append(file_path)
            except Exception as e:
                print(f"  ⚠️ Bỏ qua file lỗi: {os.path.basename(file_path)} - {e}")
        
        if not datasets:
            print(f"  ❌ Không có file hợp lệ cho tháng {month_key}")
            return False
        
        print(f"  ✓ Đọc thành công {len(datasets)} files")
        
        # Kết hợp datasets
        combined_ds = xr.concat(datasets, dim='time')
        
        # Tính trung bình theo thời gian
        monthly_mean = combined_ds.mean(dim='time', skipna=True)
        
        # Thêm thông tin metadata
        monthly_mean.attrs['title'] = f'CYGNSS Monthly Average - {month_key}'
        monthly_mean.attrs['description'] = f'Monthly average calculated from {len(datasets)} daily files'
        monthly_mean.attrs['processing_date'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        monthly_mean.attrs['source_files_count'] = len(datasets)
        
        # Tạo tên file output
        output_filename = f"CYGNSS_36km_monthly_avg_{month_key.replace('-', '_')}.nc"
        output_path = os.path.join(output_folder, output_filename)
        
        # Lưu file
        monthly_mean.to_netcdf(output_path)
        print(f"  ✅ Đã lưu: {output_filename}")
        
        # Đóng datasets để giải phóng bộ nhớ
        for ds in datasets:
            ds.close()
        
        return True
        
    except Exception as e:
        print(f"  ❌ Lỗi xử lý tháng {month_key}: {e}")
        return False

# Test với tháng đầu tiên
if sorted_months and monthly_files_dict:
    test_month = sorted_months[0]
    test_files = monthly_files_dict[test_month]
    
    print(f"🧪 Test với tháng {test_month}:")
    success = calculate_monthly_average(test_files, test_month, output_folder)

🧪 Test với tháng 2022-01:

🔄 Đang xử lý tháng 2022-01 (31 files)...
  ✓ Đọc thành công 31 files
  ✓ Đọc thành công 31 files
  ✅ Đã lưu: CYGNSS_36km_monthly_avg_2022_01.nc
  ✅ Đã lưu: CYGNSS_36km_monthly_avg_2022_01.nc


In [24]:
# Xử lý tất cả các tháng
def process_all_months(monthly_files_dict, sorted_months, output_folder):
    """
    Xử lý tất cả các tháng để tạo trung bình hàng tháng
    """
    print(f"🚀 Bắt đầu xử lý {len(sorted_months)} tháng...")
    
    successful_months = []
    failed_months = []
    
    for i, month_key in enumerate(sorted_months, 1):
        print(f"\n📅 Tiến độ: {i}/{len(sorted_months)} - Tháng {month_key}")
        
        file_list = monthly_files_dict[month_key]
        success = calculate_monthly_average(file_list, month_key, output_folder)
        
        if success:
            successful_months.append(month_key)
        else:
            failed_months.append(month_key)
    
    # Báo cáo kết quả
    print(f"\n📊 KẾT QUẢ TỔNG KẾT:")
    print(f"✅ Thành công: {len(successful_months)} tháng")
    print(f"❌ Thất bại: {len(failed_months)} tháng")
    
    if failed_months:
        print(f"\n🔍 Các tháng thất bại:")
        for month in failed_months:
            print(f"  - {month}")
    
    # Kiểm tra files đã tạo
    output_files = glob.glob(os.path.join(output_folder, "*.nc"))
    print(f"\n📁 Files đã tạo: {len(output_files)}")
    
    return successful_months, failed_months

# Lựa chọn: Xử lý tất cả hoặc chỉ một phần
print("🎯 LỰA CHỌN XỬ LÝ:")
print("1. Xử lý tất cả các tháng")
print("2. Xử lý chỉ 12 tháng đầu (test)")

# Có thể thay đổi lựa chọn ở đây
process_option = 1  # Thay đổi thành 1 để xử lý tất cả

if process_option == 1:
    # Xử lý tất cả
    successful, failed = process_all_months(monthly_files_dict, sorted_months, output_folder)
else:
    # Xử lý chỉ 12 tháng đầu
    test_months = sorted_months[:12]
    print(f"🧪 Xử lý test với {len(test_months)} tháng đầu...")
    successful, failed = process_all_months(monthly_files_dict, test_months, output_folder)

🎯 LỰA CHỌN XỬ LÝ:
1. Xử lý tất cả các tháng
2. Xử lý chỉ 12 tháng đầu (test)
🚀 Bắt đầu xử lý 36 tháng...

📅 Tiến độ: 1/36 - Tháng 2022-01

🔄 Đang xử lý tháng 2022-01 (31 files)...
  ✓ Đọc thành công 31 files
  ✓ Đọc thành công 31 files
  ✅ Đã lưu: CYGNSS_36km_monthly_avg_2022_01.nc

📅 Tiến độ: 2/36 - Tháng 2022-02

🔄 Đang xử lý tháng 2022-02 (28 files)...
  ✅ Đã lưu: CYGNSS_36km_monthly_avg_2022_01.nc

📅 Tiến độ: 2/36 - Tháng 2022-02

🔄 Đang xử lý tháng 2022-02 (28 files)...
  ✓ Đọc thành công 28 files
  ✓ Đọc thành công 28 files
  ✅ Đã lưu: CYGNSS_36km_monthly_avg_2022_02.nc

📅 Tiến độ: 3/36 - Tháng 2022-03

🔄 Đang xử lý tháng 2022-03 (31 files)...
  ✅ Đã lưu: CYGNSS_36km_monthly_avg_2022_02.nc

📅 Tiến độ: 3/36 - Tháng 2022-03

🔄 Đang xử lý tháng 2022-03 (31 files)...
  ✓ Đọc thành công 31 files
  ✓ Đọc thành công 31 files
  ✅ Đã lưu: CYGNSS_36km_monthly_avg_2022_03.nc

📅 Tiến độ: 4/36 - Tháng 2022-04

🔄 Đang xử lý tháng 2022-04 (30 files)...
  ✅ Đã lưu: CYGNSS_36km_monthly_avg_2022_0

In [25]:
# Kiểm tra và visualize kết quả
def examine_monthly_results(output_folder):
    """
    Kiểm tra các file trung bình hàng tháng đã tạo
    """
    output_files = glob.glob(os.path.join(output_folder, "*.nc"))
    
    if not output_files:
        print("❌ Không tìm thấy file kết quả nào!")
        return
    
    print(f"📊 KIỂM TRA KẾT QUẢ:")
    print(f"Số file đã tạo: {len(output_files)}")
    
    # Sắp xếp files theo tên
    output_files.sort()
    
    print(f"\n📁 Danh sách files:")
    for i, file_path in enumerate(output_files[:10], 1):  # Hiển thị 10 file đầu
        filename = os.path.basename(file_path)
        file_size = os.path.getsize(file_path) / (1024*1024)  # MB
        print(f"  {i:2d}. {filename} ({file_size:.1f} MB)")
    
    if len(output_files) > 10:
        print(f"  ... và {len(output_files) - 10} files khác")
    
    # Kiểm tra file đầu tiên
    if output_files:
        print(f"\n🔍 KIỂM TRA CHI TIẾT FILE ĐẦU TIÊN:")
        first_file = output_files[0]
        
        try:
            with xr.open_dataset(first_file) as ds:
                print(f"File: {os.path.basename(first_file)}")
                print(f"Dimensions: {dict(ds.dims)}")
                print(f"Variables: {list(ds.data_vars.keys())}")
                
                # Kiểm tra metadata
                if 'title' in ds.attrs:
                    print(f"Title: {ds.attrs['title']}")
                if 'source_files_count' in ds.attrs:
                    print(f"Source files: {ds.attrs['source_files_count']}")
                
                # Kiểm tra giá trị data
                for var_name, var in ds.data_vars.items():
                    if var.size > 0:
                        valid_data = var.where(~np.isnan(var))
                        if valid_data.size > 0:
                            print(f"{var_name}: min={float(valid_data.min()):.3f}, max={float(valid_data.max()):.3f}")
                
        except Exception as e:
            print(f"❌ Lỗi đọc file: {e}")

# Kiểm tra kết quả
examine_monthly_results(output_folder)

📊 KIỂM TRA KẾT QUẢ:
Số file đã tạo: 36

📁 Danh sách files:
   1. CYGNSS_36km_monthly_avg_2022_01.nc (9.3 MB)
   2. CYGNSS_36km_monthly_avg_2022_02.nc (9.3 MB)
   3. CYGNSS_36km_monthly_avg_2022_03.nc (9.3 MB)
   4. CYGNSS_36km_monthly_avg_2022_04.nc (9.3 MB)
   5. CYGNSS_36km_monthly_avg_2022_05.nc (9.3 MB)
   6. CYGNSS_36km_monthly_avg_2022_06.nc (9.3 MB)
   7. CYGNSS_36km_monthly_avg_2022_07.nc (9.3 MB)
   8. CYGNSS_36km_monthly_avg_2022_08.nc (9.3 MB)
   9. CYGNSS_36km_monthly_avg_2022_09.nc (9.3 MB)
  10. CYGNSS_36km_monthly_avg_2022_10.nc (9.3 MB)
  ... và 26 files khác

🔍 KIỂM TRA CHI TIẾT FILE ĐẦU TIÊN:
File: CYGNSS_36km_monthly_avg_2022_01.nc
Dimensions: {'lat': 252, 'lon': 802, 'timeslices': 4}
Variables: ['SM_daily', 'SM_subdaily', 'SIGMA_daily', 'SIGMA_subdaily']
Title: CYGNSS Monthly Average - 2022-01
Source files: 31
SM_daily: min=0.013, max=0.867
SM_subdaily: min=0.003, max=0.887
SIGMA_daily: min=0.000, max=0.323
SIGMA_subdaily: min=0.000, max=0.757
SIGMA_subdaily: min=0.

In [23]:
# Optional: Visualization nhanh (nếu có matplotlib)
def quick_visualization(output_folder):
    """
    Tạo visualization nhanh cho một file mẫu
    """
    try:
        import matplotlib.pyplot as plt
        
        output_files = glob.glob(os.path.join(output_folder, "*.nc"))
        if not output_files:
            print("Không có file để visualize")
            return
        
        # Mở file đầu tiên
        sample_file = output_files[0]
        with xr.open_dataset(sample_file) as ds:
            print(f"📈 Visualization for: {os.path.basename(sample_file)}")
            
            # Lấy biến đầu tiên có dữ liệu 2D
            for var_name, var in ds.data_vars.items():
                if len(var.dims) >= 2:
                    plt.figure(figsize=(10, 6))
                    var.plot(cmap='viridis')
                    plt.title(f'CYGNSS Monthly Average - {var_name}')
                    plt.tight_layout()
                    plt.show()
                    break
                    
    except ImportError:
        print("📊 Matplotlib không có sẵn - bỏ qua visualization")
    except Exception as e:
        print(f"❌ Lỗi visualization: {e}")

# Uncomment dòng dưới để chạy visualization
# quick_visualization(output_folder)

print("=" * 60)
print("🎉 HOÀN THÀNH QUY TRÌNH XỬ LÝ CYGNSS MONTHLY AVERAGES!")
print("=" * 60)
print()
print("📋 TÓM TẮT:")
print("✓ Đã phân tích cấu trúc dữ liệu CYGNSS")
print("✓ Đã nhóm files theo tháng")
print("✓ Đã tính toán trung bình hàng tháng")
print("✓ Đã lưu kết quả vào thư mục 'monthly_averages'")
print()
print("🔧 SỬ DỤNG:")
print(f"- Files kết quả được lưu trong: {output_folder}")
print("- Format: CYGNSS_36km_monthly_avg_YYYY_MM.nc")
print("- Có thể sử dụng các files này cho phân tích tiếp theo")
print()
print("💡 GỢI Ý:")
print("- Thay đổi 'process_option = 1' để xử lý tất cả các tháng")
print("- Kiểm tra thư mục output để xem kết quả")
print("- Có thể tùy chỉnh thêm các bước xử lý khác")

🎉 HOÀN THÀNH QUY TRÌNH XỬ LÝ CYGNSS MONTHLY AVERAGES!

📋 TÓM TẮT:
✓ Đã phân tích cấu trúc dữ liệu CYGNSS
✓ Đã nhóm files theo tháng
✓ Đã tính toán trung bình hàng tháng
✓ Đã lưu kết quả vào thư mục 'monthly_averages'

🔧 SỬ DỤNG:
- Files kết quả được lưu trong: monthly_averages
- Format: CYGNSS_36km_monthly_avg_YYYY_MM.nc
- Có thể sử dụng các files này cho phân tích tiếp theo

💡 GỢI Ý:
- Thay đổi 'process_option = 1' để xử lý tất cả các tháng
- Kiểm tra thư mục output để xem kết quả
- Có thể tùy chỉnh thêm các bước xử lý khác
